In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset, random_split
from tqdm import tqdm
from collections import defaultdict
import random

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
full_dataset = ImageFolder(root="Dataset", transform=transform)
print("Classes detected:", full_dataset.classes)

# Uniform sampling
samples_per_class = 500
class_indices = defaultdict(list)
for idx, (_, label) in enumerate(full_dataset.samples):
    class_indices[label].append(idx)

selected_indices = []
for label in class_indices:
    if len(class_indices[label]) >= samples_per_class:
        selected_indices.extend(random.sample(class_indices[label], samples_per_class))
    else:
        print(f"Warning: Not enough samples in class {full_dataset.classes[label]}")
        selected_indices.extend(class_indices[label])

# Subset and split
balanced_dataset = Subset(full_dataset, selected_indices)
val_size = int(0.2 * len(balanced_dataset))
train_size = len(balanced_dataset) - val_size
train_dataset, val_dataset = random_split(balanced_dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Model
model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, len(full_dataset.classes))
model = model.to(device)

# Loss, optimizer, scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

# Training
num_epochs = 10
best_val_loss = float('inf')
early_stopping_patience = 3
patience_counter = 0

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        progress_bar.set_postfix(loss=total_train_loss / len(train_loader))

    avg_train_loss = total_train_loss / len(train_loader)

    # Validation
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # Scheduler step
    scheduler.step()

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), "best_vgg16_model.pth")
        print("✅ Saved best model.")
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("⛔ Early stopping triggered.")
            break

print("Training complete.")


Classes detected: ['aluminium', 'brass', 'copper']


Epoch 1/10: 100%|██████████| 300/300 [09:08<00:00,  1.83s/it, loss=0.318]


Epoch [1/10] | Train Loss: 0.3179 | Val Loss: 0.1912
✅ Saved best model.


Epoch 2/10: 100%|██████████| 300/300 [09:06<00:00,  1.82s/it, loss=0.169] 


Epoch [2/10] | Train Loss: 0.1695 | Val Loss: 0.0060
✅ Saved best model.


...

Epoch [8/10] | Train Loss: 0.0000 | Val Loss: 0.0005


Epoch 9/10: 100%|██████████| 300/300 [09:10<00:00,  1.83s/it, loss=1.14e-5]


Epoch [9/10] | Train Loss: 0.0000 | Val Loss: 0.0000
✅ Saved best model.


Epoch 10/10: 100%|██████████| 300/300 [09:07<00:00,  1.83s/it, loss=2.98e-6]


Epoch [10/10] | Train Loss: 0.0000 | Val Loss: 0.0001
Training complete.
